<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong><em>Find and Delete Old EBS Snapshots</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-Old-EBS-Snapshots"><u>Delete Old EBS Snapshots</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1.&nbsp;Find Old EBS Snapshots<br>2.&nbsp;Delete old EBS snapshots</p>

In [8]:
if snapshot_ids and not region:
    raise SystemExit("Provide a region for the EBS Snapshots!")
if region == None:
    region = ""

<h3 id="Filter-unused-NAT-Gateways"><a id="1" target="_self" rel="nofollow"></a>Find Old EBS Snapshots</h3>
<p>Using unSkript's Filter AWS Find Old EBS Snapshots action, we will find old snapshots given a threshold number of days.</p>
<blockquote>
<p>This action takes the following parameters: <code>region, threhold_days</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>unused_snapshots</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint
from datetime import datetime, timedelta
import pytz


from beartype import beartype
@beartype
def aws_filter_old_ebs_snapshots_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_filter_old_ebs_snapshots(handle, region: str="", threshold: int = 30) -> Tuple:
    """aws_filter_old_ebs_snapshots Returns an array of EBS snapshots details.

        :type region: string
        :param region: AWS Region.

        :type threshold: int
        :param threshold: (in days) The threshold to check the snapshots older than the threshold.

        :rtype: List of EBS snapshots details.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            # Filtering the volume by region
            current_time = datetime.now(pytz.UTC)
            ec2Client = handle.resource('ec2', region_name=reg)
            response = ec2Client.snapshots.filter(OwnerIds=['self'])
            for snapshot in response:
                snap_data = {}
                running_time = current_time - snapshot.start_time
                if running_time > timedelta(days=int(threshold)):
                    snap_data["region"] = reg
                    snap_data["snapshot_id"] = snapshot.id
                    result.append(snap_data)
        except Exception as e:
            pass
    if len(result)!=0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "threshold": "int(threshold_days)"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not snapshot_ids",
    "condition_result": true
    }''')
task.configure(outputName="unused_snapshots")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_filter_old_ebs_snapshots, lego_printer=aws_filter_old_ebs_snapshots_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Unused-NAT-Gateways">Create List of Old EBS Snapshots</h3>
<p>This action filters regions that have no old EBS snapshots and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_unused_snapshots</code></p>
</blockquote>

In [ ]:
all_unused_snapshots = []
dummy = []
try:
    if unused_snapshots[0] == False:
        for snapshot in unused_snapshots[1]:
            all_unused_snapshots.append(snapshot)
except Exception as e:
    if snapshot_ids:
        for snap in snapshot_ids:
            data_dict = {}
            data_dict["region"] = region
            data_dict["snapshot_id"] = snap
            all_unused_snapshots.append(data_dict)
    else:
         raise Exception(e)

<h3 id="Delete-unused-NAT-Gateways"><a id="2" target="_self" rel="nofollow"></a>Delete old EBS Snapshots</h3>
<p>This action deletes old EBS Snapshots found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>region, snapshot_id</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Dict
import pprint

from beartype import beartype
@beartype
def aws_delete_ebs_snapshot_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_ebs_snapshot(handle, region: str, snapshot_id: str) -> Dict:
    """aws_delete_ebs_snapshot Returns a dict of deleted snapshot details

        :type region: string
        :param region: AWS Region.

        :type snapshot_id: string
        :param snapshot_id: EBS snapshot ID. Eg: 'snap-34bt4bfjed9d'

        :rtype: Deleted snapshot details
    """
    result = []
    try:
        ec2Client = handle.client('ec2', region_name=region)
        result = ec2Client.delete_snapshot(SnapshotId=snapshot_id)
    except Exception as e:
        raise e
    return  result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "snapshot_id": "iter.get(\\"snapshot_id\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_unused_snapshots",
    "iter_parameter": ["region","snapshot_id"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_unused_snapshots)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_ebs_snapshot, lego_printer=aws_delete_ebs_snapshot_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;">Conclusion</h3>
<p>In this Runbook, we were able to filter old EBS Snapshots given a threshold number of days and delete them. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>